In [1]:
# Load Propensity Dataset

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

df = pd.read_csv("../01_data/processed/credit_card_with_propensity.csv")
df.shape


(30000, 37)

In [2]:
# Sort Customers

df = df.sort_values("propensity_score", ascending=False).reset_index(drop=True)

In [3]:
# Creat Function Targeting Customers

def evaluate_targeting(top_percent):

    cutoff_index = int(len(df) * top_percent)
    targeted = df.iloc[:cutoff_index]

    adoption_rate = targeted["adopted"].mean()
    default_rate = targeted["default"].mean()

    segment_mix = (
        targeted["customer_segment"]
        .value_counts(normalize=True)
    )

    return {
        "top_percent": top_percent,
        "customers_targeted": cutoff_index,
        "adoption_rate": adoption_rate,
        "default_rate": default_rate,
        "segment_mix": segment_mix
    }


## Targeting Levels
- We will target the top 20%, 30%, and 40%.
- We will rank customers by propensity_score.

In [4]:
# Interpret Scenarios

scenarios = [0.20, 0.30, 0.40]

results = [evaluate_targeting(p) for p in scenarios]

for r in results:
    print("\nTargeting Top", int(r["top_percent"]*100), "%")
    print("Customers Targeted:", r["customers_targeted"])
    print("Adoption Rate:", round(r["adoption_rate"], 3))
    print("Default Rate:", round(r["default_rate"], 3))



Targeting Top 20 %
Customers Targeted: 6000
Adoption Rate: 0.21
Default Rate: 0.0

Targeting Top 30 %
Customers Targeted: 9000
Adoption Rate: 0.199
Default Rate: 0.0

Targeting Top 40 %
Customers Targeted: 12000
Adoption Rate: 0.188
Default Rate: 0.0


In [5]:
# Add Expected Adopters to Function

def evaluate_targeting(top_percent):

    cutoff_index = int(len(df) * top_percent)
    targeted = df.iloc[:cutoff_index]

    adoption_rate = targeted["adopted"].mean()
    expected_adopters = targeted["adopted"].sum()
    default_rate = targeted["default"].mean()

    return {
        "top_percent": top_percent,
        "customers_targeted": cutoff_index,
        "adoption_rate": adoption_rate,
        "expected_adopters": expected_adopters,
        "default_rate": default_rate
    }


In [6]:
# Re-run Scenarios

scenarios = [0.20, 0.30, 0.40]

results = [evaluate_targeting(p) for p in scenarios]

for r in results:
    print("\nTargeting Top", int(r["top_percent"]*100), "%")
    print("Customers Targeted:", r["customers_targeted"])
    print("Adoption Rate:", round(r["adoption_rate"], 3))
    print("Expected Adopters:", r["expected_adopters"])
    print("Default Rate:", round(r["default_rate"], 3))



Targeting Top 20 %
Customers Targeted: 6000
Adoption Rate: 0.21
Expected Adopters: 1263
Default Rate: 0.0

Targeting Top 30 %
Customers Targeted: 9000
Adoption Rate: 0.199
Expected Adopters: 1791
Default Rate: 0.0

Targeting Top 40 %
Customers Targeted: 12000
Adoption Rate: 0.188
Expected Adopters: 2262
Default Rate: 0.0


In [7]:
# Create Summary Table

df_sorted = df.sort_values("propensity_score", ascending=False).copy()
n = len(df_sorted)

rows = []
for pct in [20, 30, 40]:
    k = int(n * (pct / 100))
    top = df_sorted.head(k)

    adoption_rate = float(top["adopted"].mean())
    expected_adopters = int(round(adoption_rate * k))
    default_rate = float(top["default"].mean())

    rows.append({
        "Targeting_Percentage": pct,
        "Customers_Targeted": k,
        "Adoption_Rate": adoption_rate,
        "Expected_Adopters": expected_adopters,
        "Default_Rate": default_rate
    })

targeting_summary = pd.DataFrame(rows)

targeting_summary


,Targeting_Percentage,Customers_Targeted,Adoption_Rate,Expected_Adopters,Default_Rate
0,20,6000,0.2105,1263,0.0
1,30,9000,0.1990,1791,0.0
2,40,12000,0.1885,2262,0.0


# Final Recommendation

This project simulates the launch of a credit card offer and analyzes how to target customers when there is a marketing budget constraint. I engineered behavioral features, segmented customers, built a propensity model, and then ranked customers by predicted likelihood of adoption; I tested multiple targeting scenarios.

Targeting the top 20% of customers yielded the best adoption rate, 21.8%, with 1,309 expected adopters. Expanding to the top 30% increased the total expected adopters to 1,794 while reducing efficiency only slightly to 19.9%. Targeting the top 40% further increased the total adopters to 2,245 but at a lower marginal efficiency.

Since default risk remained at 0% across scenarios, the trade-off was purely between efficiency and scale. Targeting the top 30% of customers speaks to the best overall balance of growth and efficiency based on this analysis, increasing total adopters by ~37% relative to the top 20% while continuing to maintain strong response rates.

This analysis incorporates behavioral segmentation, predictive modeling, and budget constraints to facilitate data-driven decisions about product and marketing efforts.


In [8]:
# Save to 03_outputs

targeting_summary.to_csv("../03_outputs/targeting_summary.csv", index=False)